# Movie Review Text Classification
This tutorial https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [3]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.2.0-dev20200429
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [4]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMROA0G/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMROA0G/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMROA0G/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(5)))
train_examples_batch, train_labels_batch

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
        b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fel

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 2s 72ms/step - loss: 0.6717 - accuracy: 0.5854 - val_loss: 0.6638 - val_accuracy: 0.6120
Epoch 2/20
30/30 [==============================] - 2s 72ms/step - loss: 0.6608 - accuracy: 0.6277 - val_loss: 0.6552 - val_accuracy: 0.6548
Epoch 3/20
30/30 [==============================] - 2s 74ms/step - loss: 0.6516 - accuracy: 0.6647 - val_loss: 0.6460 - val_accuracy: 0.6744
Epoch 4/20
30/30 [==============================] - 2s 69ms/step - loss: 0.6414 - accuracy: 0.6935 - val_loss: 0.6355 - val_accuracy: 0.7055
Epoch 5/20
30/30 [==============================] - 2s 67ms/step - loss: 0.6288 - accuracy: 0.7323 - val_loss: 0.6235 - val_accuracy: 0.7464
Epoch 6/20
30/30 [==============================] - 2s 70ms/step - loss: 0.6158 - accuracy: 0.7662 - val_loss: 0.6132 - val_accuracy: 0.7755
Epoch 7/20
30/30 [==============================] - 2s 70ms/step - loss: 0.6046 - accuracy: 0.7933 - val_loss: 0.6046 - val_accuracy: 0.7833
Epoch 8/20
30

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.5714 - accuracy: 0.8520
loss: 0.571
accuracy: 0.852


In [13]:
np.set_printoptions(suppress=True)
(review, label) = next(iter(test_data.batch(20)))
prediction = model.predict(review)
(prediction, label)

(array([[0.1721141 ],
        [0.917421  ],
        [0.01723057],
        [0.00043312],
        [0.99976087],
        [0.9999311 ],
        [0.999989  ],
        [0.9993308 ],
        [0.6404022 ],
        [0.07469702],
        [0.0000003 ],
        [0.0022535 ],
        [0.24444205],
        [0.39690885],
        [0.99969184],
        [0.42759222],
        [0.7785633 ],
        [0.00051631],
        [0.99172324],
        [0.00020682]], dtype=float32),
 <tf.Tensor: shape=(20,), dtype=int64, numpy=array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0])>)